# Stratégie d'Investissement Core-Satellite : Exemple d'Implémentation

Ce notebook démontre l'utilisation du framework d'optimisation Core-Satellite pour construire et analyser un portefeuille d'investissement combinant:
- Un **noyau (core)** : Composé d'investissements passifs à faible coût- Des **satellites** : Composés d'investissements actifs ciblés

Nous allons explorer:
1. Le chargement et la préparation des données2. L'optimisation du portefeuille core3. L'optimisation des satellites4. La détermination du ratio optimal entre core et satellites5. L'analyse et la visualisation des résultats6. Le backtesting de la stratégie

In [ ]:
# Importation des librairies nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# Configurer la visualisation
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('tab10')
warnings.filterwarnings('ignore')  # Ignorer les avertissements pour la lisibilité

# Importer nos modules personnalisés
import sys
sys.path.append('..')

from src.data_processing.data_loader import DataLoader
from src.optimization.core_satellite import CoreSatelliteOptimizer
from src.visualization.portfolio_visualizer import PortfolioVisualizer
from src.backtesting.core_satellite_backtest import CoreSatelliteBacktest

## 1. Chargement et Préparation des Données

Commençons par charger les données historiques de prix pour les actifs que nous allons utiliser dans notre portefeuille. Nous allons définir:
- Des ETF pour le core (portefeuille passif)
- Des actifs individuels pour les satellites (stratégies actives)

In [ ]:
# Définir la période d'analyse
start_date = '2018-01-01'
end_date = '2023-12-31'

# Initialiser le chargeur de données
data_loader = DataLoader(start_date=start_date, end_date=end_date)

# Définir les actifs pour le core (ETF et indices)
core_tickers = [
    'SPY',   # S&P 500 ETF
    'QQQ',   # NASDAQ 100 ETF
    'EFA',   # MSCI EAFE (International Developed Markets)
    'EEM',   # MSCI Emerging Markets
    'AGG',   # US Aggregate Bond ETF
    'LQD',   # Corporate Bond ETF
    'GLD'    # Gold ETF
]

# Définir les actifs pour les satellites (actions individuelles, secteurs, etc.)
satellite_tickers = [
    'AAPL',  # Apple
    'MSFT',  # Microsoft
    'GOOGL', # Alphabet (Google)
    'AMZN',  # Amazon
    'TSLA',  # Tesla
    'XLF',   # Financial Sector ETF
    'XLV',   # Healthcare Sector ETF
    'XLE',   # Energy Sector ETF
    'ARKK',  # ARK Innovation ETF
    'JETS'   # Airline ETF
]

# Charger les données de prix pour tous les actifs
all_tickers = core_tickers + satellite_tickers
prices, returns = data_loader.get_asset_data(all_tickers)

# Afficher les premières lignes des données de rendement
returns.head()

In [ ]:
# Calculer les statistiques principales des actifs
asset_stats = data_loader.get_asset_stats(returns)

# Afficher les statistiques triées par ratio de Sharpe
asset_stats.sort_values('Sharpe Ratio', ascending=False)

In [ ]:
# Visualiser la matrice de corrélation
corr_matrix = data_loader.get_correlations(returns)

# Créer une heatmap de la matrice de corrélation
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
heatmap = sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
                    vmin=-1, vmax=1, center=0, square=True, linewidths=.5)
plt.title('Matrice de Corrélation des Actifs', fontsize=15)
plt.show()

## 2. Visualisation de la Frontière Efficiente

Avant d'optimiser notre portefeuille Core-Satellite, visualisons la frontière efficiente pour comprendre l'univers d'investissement disponible.

In [ ]:
# Initialiser le visualiseur de portefeuille
visualizer = PortfolioVisualizer()

# Tracer la frontière efficiente pour le core
visualizer.plot_efficient_frontier(
    returns[core_tickers], 
    num_portfolios=2000, 
    risk_free_rate=0.02,
    show_assets=True
)
plt.title('Frontière Efficiente - Core Assets')
plt.show()

# Tracer la frontière efficiente pour les satellites
visualizer.plot_efficient_frontier(
    returns[satellite_tickers], 
    num_portfolios=2000, 
    risk_free_rate=0.02,
    show_assets=True
)
plt.title('Frontière Efficiente - Satellite Assets')
plt.show()

## 3. Optimisation du Portefeuille Core-Satellite

Maintenant, optimisons notre portefeuille Core-Satellite en utilisant notre module d'optimisation.

In [ ]:
# Séparer les rendements entre core et satellites
core_returns = returns[core_tickers]
satellite_returns = returns[satellite_tickers]

# Initialiser l'optimiseur Core-Satellite
optimizer = CoreSatelliteOptimizer(core_returns, satellite_returns)

# Optimiser l'allocation du core
core_weights = optimizer.optimize_core_allocation(risk_aversion=2.0)
print("Allocation optimale du core:")
print(core_weights)

# Optimiser l'allocation des satellites
satellite_weights = optimizer.optimize_satellite_allocation(risk_aversion=2.0)
print("
Allocation optimale des satellites:")
print(satellite_weights)

# Optimiser le ratio entre core et satellites
core_ratio = optimizer.optimize_core_satellite_ratio(
    core_weights, satellite_weights, risk_aversion=2.0, min_core=0.3, max_core=0.9
)
print(f"
Ratio optimal core-satellites: {core_ratio:.2%} core, {1-core_ratio:.2%} satellites")

# Obtenir le portefeuille optimal complet
metrics, all_weights = optimizer.get_optimal_portfolio(
    risk_aversion=2.0, min_core=0.3, max_core=0.9
)

print("
Métriques du portefeuille optimal:")
for key, value in metrics.items():
    if key != 'risk_contributions':
        print(f"{key}: {value:.4f}")

print("
Répartition finale du portefeuille:")
for ticker, weight in all_weights.items():
    if weight > 0.001:  # Afficher uniquement les poids significatifs
        print(f"{ticker}: {weight:.2%}")

## 4. Visualisation de l'Allocation Core-Satellite

Visualisons maintenant l'allocation optimale du portefeuille Core-Satellite.

In [ ]:
# 1. Visualiser l'allocation globale
visualizer.plot_allocations(
    weights=all_weights,
    title='Allocation Globale du Portefeuille',
    kind='pie',
    core_satellite_split={'core': core_tickers, 'satellites': satellite_tickers}
)
plt.show()

# 2. Visualiser l'allocation par type (core vs satellite)
final_core_weights = core_weights * metrics['core_proportion']
final_satellite_weights = satellite_weights * (1 - metrics['core_proportion'])

visualizer.plot_core_satellite_structure(
    core_weights=pd.Series(core_weights),
    satellite_weights=pd.Series(satellite_weights),
    core_ratio=metrics['core_proportion']
)
plt.show()

# 3. Visualiser les contributions au risque
all_returns = pd.concat([core_returns, satellite_returns], axis=1)
cov_matrix = all_returns.cov()

visualizer.plot_risk_contribution(
    weights=all_weights,
    cov_matrix=cov_matrix,
    title='Contribution au Risque par Actif'
)
plt.show()

## 5. Backtesting de la Stratégie Core-Satellite

Testons maintenant la performance de notre stratégie Core-Satellite sur les données historiques avec un rebalancement périodique.

In [ ]:
# Initialiser le backtest
backtest = CoreSatelliteBacktest(
    prices=prices,
    lookback_window=252,  # 1 an de données pour l'optimisation
    rebalance_freq='Q'    # Rebalancement trimestriel
)

# Exécuter le backtest
backtest_results = backtest.run_backtest(
    optimizer_func=CoreSatelliteOptimizer,
    core_tickers=core_tickers,
    satellite_tickers=satellite_tickers,
    initial_capital=10000.0,
    start_date='2019-01-01',  # Commencer après la période de lookback
    end_date=end_date,
    risk_aversion=2.0,
    min_core_ratio=0.3,
    max_core_ratio=0.9,
    transaction_costs=0.001  # 0.1% de coûts de transaction
)

# Afficher les métriques de performance
performance_metrics = backtest.get_performance_metrics()
performance_metrics

In [ ]:
# Visualiser l'évolution du portefeuille
backtest.plot_portfolio_evolution(include_benchmarks=True)
plt.show()

# Visualiser l'évolution des poids
backtest.plot_weights_evolution(separate_core_satellite=True)
plt.show()

# Visualiser l'évolution du ratio core-satellite
backtest.plot_core_satellite_ratio()
plt.show()

## 6. Analyse des Performances Glissantes

Analysons maintenant les performances glissantes pour évaluer la stabilité de notre stratégie.

In [ ]:
# Créer un dictionnaire des rendements pour comparaison
portfolio_returns = backtest_results['portfolio_returns']

# Aligner les dates pour les benchmarks
all_core_returns = backtest_results['all_core_index'].pct_change().dropna()
all_satellite_returns = backtest_results['all_satellite_index'].pct_change().dropna()

# S'assurer que toutes les séries ont les mêmes dates
common_dates = portfolio_returns.index.intersection(all_core_returns.index).intersection(all_satellite_returns.index)
portfolio_returns = portfolio_returns.loc[common_dates]
all_core_returns = all_core_returns.loc[common_dates]
all_satellite_returns = all_satellite_returns.loc[common_dates]

returns_dict = {
    'Core-Satellite': portfolio_returns,
    'All Core': all_core_returns,
    'All Satellite': all_satellite_returns
}

# Visualiser la comparaison des performances
visualizer.plot_performance_comparison(
    returns_dict=returns_dict,
    benchmark_name='All Core',
    cumulative=True,
    risk_metrics=True
)
plt.show()

# Visualiser les performances glissantes
visualizer.plot_rolling_performance(
    returns_dict=returns_dict,
    window=60,  # 60 jours glissants
    metrics=['returns', 'volatility', 'sharpe']
)
plt.show()

## 7. Analyse de Scénarios et Tests de Stress

Analysons comment notre portefeuille Core-Satellite se comporte dans différents scénarios de marché.

In [ ]:
# Identifier les périodes de crise ou d'intérêt particulier
# Par exemple, la période COVID de 2020
covid_start = '2020-02-15'
covid_end = '2020-04-15'

covid_returns = {
    'Core-Satellite': returns_dict['Core-Satellite'].loc[covid_start:covid_end],
    'All Core': returns_dict['All Core'].loc[covid_start:covid_end],
    'All Satellite': returns_dict['All Satellite'].loc[covid_start:covid_end]
}

# Visualiser les performances pendant la crise
visualizer.plot_performance_comparison(
    returns_dict=covid_returns,
    benchmark_name='All Core',
    cumulative=True,
    risk_metrics=True,
    log_scale=False
)
plt.title('Performance pendant la Crise COVID (Fév-Avr 2020)')
plt.show()

# Calculer les drawdowns pour analyser les pertes maximales
def calculate_drawdowns(returns_series):
    wealth_index = (1 + returns_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    return drawdowns

drawdowns = {}
for name, returns_series in returns_dict.items():
    drawdowns[name] = calculate_drawdowns(returns_series)

# Tracer les drawdowns
plt.figure(figsize=(12, 6))
for name, dd_series in drawdowns.items():
    plt.plot(dd_series, label=name)

plt.title('Drawdowns des Stratégies')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Conclusions et Recommandations

Sur la base de notre analyse, voici les principales conclusions et recommandations pour la stratégie Core-Satellite :

1. **Allocation optimale** : Le ratio optimal entre le core et les satellites est de XXX% pour le core et XXX% pour les satellites, ce qui offre un bon équilibre entre stabilité et potentiel de croissance.

2. **Performance** : La stratégie Core-Satellite a [surperformé/sous-performé] les benchmarks sur la période analysée, avec un rendement annualisé de XXX% contre XXX% pour un portefeuille tout-core.

3. **Risque** : La diversification entre core et satellites a permis de réduire la volatilité globale du portefeuille à XXX%, par rapport à XXX% pour un portefeuille tout-satellite.

4. **Comportement en période de crise** : Pendant la crise du COVID, la stratégie a montré [une meilleure/une moins bonne] résilience que les benchmarks, avec un drawdown maximal de XXX% contre XXX% pour le benchmark.

5. **Recommandations de rebalancement** : Un rebalancement trimestriel semble optimal pour maintenir l'équilibre risque-rendement souhaité tout en limitant les coûts de transaction.

6. **Perspectives futures** : Considérant l'environnement de marché actuel, nous recommandons [d'augmenter/de réduire] légèrement l'allocation aux satellites pour [raisons].

Cette stratégie Core-Satellite offre un cadre flexible qui peut être adapté en fonction de l'évolution des conditions de marché et des objectifs d'investissement spécifiques.